In [6]:
import numpy as np
import pandas as pd

from statsmodels.stats.weightstats import ztest
from statsmodels.stats.multitest import multipletests

In [2]:
data = pd.read_csv("./data/free_throws.csv")

In [3]:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


Здесь нас интересуют следующие колонки:

player — имя игрока

playoffs — этап сезона NBA (regular/playoffs)

shot_made — попал или не попал игрок штрафной бросок

Теперь будем для каждого игрока считать его среднюю результативность (долю попаданий) во время регулярного чемпионата и плей-офф, p_value, полученное через z-test, и запишем это в новый датафрейм. Z-test используем потому, что в данном случае наша задача является аналогом конверсии, например, в интернет-магазине, то есть мы имеем дело с распределением Бернулли.

Также мы оставим только тех игроков, у которых есть хотя бы по 30 бросков в регулярном сезоне и плей-офф. Это делаем для более корректной оценки, чтобы не считать тех игроков, у которых слишком мало бросков. Цифра 30 исходит из закона больших чисел, если коротко — такого количества наблюдений достаточно, чтобы довольно точно аппроксимировать нормальное распределение.

In [7]:
new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
        
    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

Посмотрим на 10 случайных элементов, чтобы увидеть, что у нас получилось:

In [9]:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
94,Harrison Barnes,0.737805,0.756303,0.672233
116,Jeremy Lin,0.802450,0.824561,0.681169
198,Patrick Patterson,0.749280,0.833333,0.201785
203,Paul Pierce,0.827090,0.835347,0.604012
194,Norris Cole,0.743386,0.782609,0.563859
42,Caron Butler,0.853771,0.850000,0.917439
40,Carlos Boozer,0.714034,0.725714,0.652178
242,Taj Gibson,0.692565,0.691176,0.973500
37,C.J. Miles,0.799242,0.854167,0.350927
243,Tayshaun Prince,0.745775,0.738739,0.870134


Теперь посчитаем количество игроков с p_value < 0.05. Так мы поймём, для скольких игроков мы бы отвергли нулевую гипотезу без поправок о множественной проверки гипотез.

In [11]:
new_df.query("p_value < 0.05").shape

(22, 4)

Расчёты готовы, и нам необходимо сравнить их с методами, изученными в предыдущем юните. 

Cравним результаты с поправкой Бонферрони и методом Холма:



In [14]:
multipletests(
    pvals=new_df.p_value,
    alpha=0.05,
    method="bonferroni"
)[0].sum()

1

In [15]:
multipletests(
    pvals=new_df.p_value,
    alpha=0.05,
    method="holm"
)[0].sum()

1

Видим, что в данной задаче два метода выдали одинаковое количество игроков, для которых стоит отвергнуть нулевую гипотезу. То есть тех, у кого с 95 % отличается результативность во время плей-офф и регулярного сезона.

Однако мы видим, насколько данное число отличается от того, что мы получили без поправок на множественную проверку гипотез. Так нам удалось кратно уменьшить вероятность ошибок первого рода.